# EEG Pipeline - Summary

WIP:
- Perplexity project: https://www.perplexity.ai/search/i-am-building-a-pipeline-in-py-sVmw70rHQXa0VKkIT_p5tg?0=r
- See Perplex notebooks, .md downloaded
- Python Handbook: https://github.com/ZitongLu1996/Python-EEG-Handbook/tree/master

MNE vs NueroDSP:
- Use MNE
- https://www.perplexity.ai/search/i-am-building-a-pipeline-in-py-sVmw70rHQXa0VKkIT_p5tg?0=d#1


# Packages & Utilities

## Import:
- MNE-Python: https://mne.tools/stable/index.html
- The Brain Imaging Data Structure (BIDS): https://bids.neuroimaging.io
- see example using mne https://neurodsp-tools.github.io/neurodsp/auto_examples/plot_mne_example.html#sphx-glr-auto-examples-plot-mne-example-py

## Spectral Parameterisation:
- SpecParam / FOOOF
- The Aperiodic Methods project - Documentation: https://aperiodicmethods.github.io/docs/index.html and Repo: in https://github.com/AperiodicMethods/AperiodicMethods
- And cite: https://www.biorxiv.org/content/10.1101/2024.09.15.613114v1

Examples
- Explanation of FOOOFing and calculation: https://fooof-tools.github.io/fooof/auto_tutorials/plot_01-ModelDescription.html#sphx-glr-auto-tutorials-plot-01-modeldescription-py


In [1]:
# General imports
import os
import gc
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# MNE-Python
import mne

# FOOF
from fooof import FOOOF
from fooof import FOOOFGroup
from fooof.plts.spectra import plot_spectra
# from fooof.plts.annotate import plot_annotated_model
# from fooof.analysis import get_band_peak_fm

# Import functions from specparam(fooof)
# from fooof.utils import trim_spectrum

# Check the version of the module
from fooof import __version__ as fooof_version
print('Current fooof version:', fooof_version)

/var/folders/wd/rhzj_w8570g_y66t6j6mh4zw0000gn/T/ipykernel_89708/2644039310.py:15: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  from fooof import FOOOF


Current fooof version: 1.1.1


In [2]:
# A simple utility function to establish paths for data files
def get_data_path(data_directory = 'Data', file_name = ''):
    project_root = os.path.dirname(os.getcwd())
    data_path = os.path.join(project_root, data_directory, file_name)
    if not os.path.isdir(data_path):
        raise Exception(f'Directory not found: {data_path}')  
     
    return data_path

# Utility function to check for the existence of a file in a given directory
def get_file_path(directory, file_name):
    file_path = os.path.join(directory, file_name)
    if not os.path.isfile(file_path):
        raise Exception(f'File not found: {file_path}')
    return file_path

## Setup & Defintions

In [3]:
# Assumed OpenNeuro, BIDS compliant datasets manually downloaded into the defined folders structure

# Definitions for this project
notebooks_folder = 'Full_Pipeline'
eeg_datasets_source = 'EEG_Datasets_Source_x'

# Create & Check Paths
notebooks_path = get_data_path(notebooks_folder)
eeg_datasets_path = get_data_path(eeg_datasets_source)

print(f'Check On Folders & Files Used')
print(f'Notebooks Folder: {notebooks_path}')
print(f'EEG Source Datasets Folder: {eeg_datasets_path}')

# Get a list of datasets (directories/files) in the EEG datasets source folder
datasets_list = os.listdir(eeg_datasets_path)
datasets_list = [d for d in datasets_list if d.startswith('ds') and os.path.isdir(os.path.join(eeg_datasets_path, d))]
print('Datasets found:', datasets_list)
# !ls "{eeg_datasets_path}"

del eeg_datasets_source, notebooks_folder
# gc.collect()

Check On Folders & Files Used
Notebooks Folder: /Users/stuartgow/GitHub/AI_Dissertation/Full_Pipeline/
EEG Source Datasets Folder: /Users/stuartgow/GitHub/AI_Dissertation/EEG_Datasets_Source_x/
Datasets found: ['ds004584-1.0.0']


## EEG Data Load & Import


In [4]:
# Function to get the raw EEG signal data
def get_EEG_raw(dataset_name, subject_id, verbose=False):
    """
    Get raw EEG data.
    Parameters:
        dataset_name - str
        subject_id - str
    Returns:
        EEG - raw
    """

    # BIDS File Structure
    dataset_root = eeg_datasets_path
    dataset_name = dataset_name
    subject = subject_id
    session = ''
    task = 'Rest'
    datatype='eeg'

    # EEGLab .set file name
    temp_path = os.path.join(dataset_root, dataset_name, subject, session, datatype)
    temp_file_name = subject + '_task-' + task + '_' + datatype + '.set'
    eeg_lab_file_path = get_file_path(temp_path, temp_file_name)

    # Get the raw EEG data & Inspect it
    eeg_dataset_raw = mne.io.read_raw_eeglab(eeg_lab_file_path, preload=True)

    if verbose:
        print(f"Description: {eeg_dataset_raw.info['description']} on {eeg_dataset_raw.info['meas_date']}")
        print(eeg_dataset_raw)
        print(eeg_dataset_raw.info)
        # print(eeg_dataset_raw.info['sfreq'])

        # Plot Time Series - MNE wbdwindowow plot
        # eeg_dataset_raw.plot(duration=10, n_channels=16, clipping=None)
        # TODO: Add an inline plot of all the channels time series ... NeuroDSP if not MNE

        # Settings for exploring an example channel of data
        fs = eeg_dataset_raw.info['sfreq']
        ch_label = 'P5'
        t_start = 0
        t_stop = int(t_start + (10 * fs))

        # Extract an example channel
        sig, times = eeg_dataset_raw.get_data(mne.pick_channels(eeg_dataset_raw.ch_names, [ch_label]),
                                start=t_start, stop=t_stop,
                                return_times=True)
        sig = np.squeeze(sig)

        plt.figure(figsize=(12, 4))
        plt.plot(times, sig, color='blue')
        plt.title(f"EEG Time Series - Channel {ch_label}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude (uV)")
        plt.tight_layout()
        plt.show()

        # Average PSD All Channels
        eeg_dataset_raw.compute_psd().plot(average=True)
        eeg_dataset_raw.compute_psd().plot()
        plt.show()

    return eeg_dataset_raw

## Power Spectrum, PSD Calculate

In [5]:
# Function to Calculate the Power Spectrum - MNE
def get_spectra(eeg_raw, verbose=False):
    """
    Calculate the Power Spectra for all channels
    Parameters:
        eeg_raw - raw
    Returns:
        Spectra - frequencies and powers for each channel
    """

    method = 'welch'
    fmin = 1
    fmax = np.inf

    print(f"Computing power spectrum using {method} method...")
    spectra = eeg_raw.compute_psd(method=method, fmin=fmin, fmax=fmax)
    
    if verbose:
        print(spectra.info)

        x_freqs = spectra.freqs
        x_powers = spectra.get_data(return_freqs=False)

        # Log-log plot for all channels
        plt.figure(figsize=(12, 6))
        for idx, ch in enumerate(eeg_raw.ch_names):
            plt.loglog(x_freqs, x_powers[idx], alpha=0.5, label=ch if idx < 10 else None)  # label only first 10 for clarity
        plt.title(f"Power Spectrum (Log-Log) - All Channels ({method} method)")
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Power ($V^2/Hz$)")
        plt.tight_layout()
        plt.show()

        # Log-lin plot for all channels
        plt.figure(figsize=(12, 6))
        for idx, ch in enumerate(eeg_raw.ch_names):
            plt.semilogy(x_freqs, x_powers[idx], alpha=0.5, label=ch if idx < 10 else None)  # label only first 10 for clarity
        plt.title(f"Power Spectrum (Log-Lin) - All Channels ({method} method)")
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Power ($V^2/Hz$)")
        plt.tight_layout()
        plt.show()

        # A selected channel
        ch_label = 'P5'
        fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
        fig.suptitle(f'Power Spectrum - Channel {ch_label} ({method} method)', fontsize=20)
        ax0.set_title('Log/Log')
        ax1.set_title('Log/Lin')
        plot_spectra(x_freqs, x_powers[eeg_raw.ch_names.index(ch_label)], log_freqs=True, log_powers=True, ax=ax0, colors='blue')
        plot_spectra(x_freqs, x_powers[eeg_raw.ch_names.index(ch_label)], log_powers=True, ax=ax1, colors='blue')

        plt.show()
    
    return spectra


## Spectral Parameterisation


In [6]:
# Function to perform Spectral Parameterisation and return the Aperiodic and Periodic components
def get_specparams(spectra, verbose=False):
    """
    Execute Spectral Parameterisation
    Parameters:
        xxxx
    Returns:
        Specparam features - dict of aperiodic and periodic components
    """
    # Parameters for SpecParam
    max_peak_width = 12
    min_peak_height = 0.05
    max_n_peaks = 25
    fit_freq_range = [1, 75]
    aperiodic_mode = 'fixed'

    # For group of spectra
    freqs = spectra.freqs
    powers = spectra.get_data(return_freqs=False)
    n_channels = powers.shape[0]
    n_frequencies = powers.shape[1]
    freq_resolution = freqs[1] - freqs[0]
    print(f'For Spectra Group:')
    print(f'- {n_channels} channels & {n_frequencies} frequencies')
    print(f'- Frequency Range: {freqs.min():.0f} to {freqs.max():.0f} Hz')
    print(f'- Frequency resolution {freq_resolution:.2f} Hz')

    peak_width_limits = [int(math.ceil(2 * freq_resolution)), max_peak_width]
    print('Extracting Periodic & Aperiodic Components with Parameters:')
    print(f'- Peak width limits: [{peak_width_limits[0]:.0f}, {peak_width_limits[1]:.0f}]')
    print(f'- Min peak height: {min_peak_height:.2f}')
    print(f'- Max peaks: {max_n_peaks}')
    print(f'- Range: {fit_freq_range}')
    print(f'- Aperiodic Mode: {aperiodic_mode}')

    # Initialize FOOOF model
    fg = FOOOFGroup(peak_width_limits=peak_width_limits, 
                    max_n_peaks=max_n_peaks, min_peak_height=min_peak_height,
                    aperiodic_mode=aperiodic_mode)

    # Fit the group of n spectra
    fg.fit(freqs, powers, fit_freq_range, progress='tqdm.notebook')

    # Get the component parameters and create a features dataframe
    aperiodics = fg.get_params('aperiodic_params')
    periodics = fg.get_params('peak_params')

    specparam_features = {}
    for next_channel in range(aperiodics.shape[0]):
        specparam_features[f'ch{next_channel}_aperiodic_offset'] = aperiodics[next_channel, 0]
        specparam_features[f'ch{next_channel}_aperiodic_exponent'] = aperiodics[next_channel, 1]

        # For up to the max peaks, get periodic components
        peaks_ch = periodics[periodics[:, 3] == next_channel]
        peaks_found = peaks_ch.shape[0]
        if peaks_found > max_n_peaks:
            print(f'WARNING: Peaks found: {peaks_found} but only {max_n_peaks} saved')
        for next_peak in range(max_n_peaks):
            if peaks_found > next_peak:
                specparam_features[f'ch{next_channel}_pk{next_peak}_periodic_CF'] = peaks_ch[next_peak, 0]
                specparam_features[f'ch{next_channel}_pk{next_peak}_periodic_PW'] = peaks_ch[next_peak, 1]
                specparam_features[f'ch{next_channel}_pk{next_peak}_periodic_BW'] = peaks_ch[next_peak, 2]
            else:
                specparam_features[f'ch{next_channel}_pk{next_peak}_periodic_CF'] = float('nan')
                specparam_features[f'ch{next_channel}_pk{next_peak}_periodic_PW'] = float('nan')
                specparam_features[f'ch{next_channel}_pk{next_peak}_periodic_BW'] = float('nan')


    # Examine results if verbose
    if verbose:
        # Overall group results
        fg.print_results()
        fg.plot()
        plt.show()

        # Visualise the fit for a sample single channel within the group
        channel_indx = 1
        ch_label = str(channel_indx)
        # TODO: Proper lable lookup
        # ch_label = eeg_dataset_raw.ch_names[channel_indx]

        fm = fg.get_fooof(ind=channel_indx, regenerate=True)
        fm.print_results()

        # fm.plot()
        fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
        fig.suptitle(f'Spec Param for Channel: {ch_label}', fontsize=20)
        ax0.set_title('Components - Log/Lin')
        ax1.set_title('Components - Log/Log')
        fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'}, ax=ax0)
        fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'}, plt_log=True, ax=ax1)
        plt.show()

    return specparam_features


# EEG Dataset Processing Pipeline

- For a defined study with subject details and raw EEG signal data
- Produce a features dataframe with Spectral Parameterisation results, ie Aperiodic and Periodic components

In [7]:
# End to end pipeline for all datasets
# Creates a dataframe with subject details and EEG specparam results for all channels

# Resulting dataframe with subject details and EEG periodic, aperiodic parameters
study_features_df = pd.DataFrame()

# Iterate through each datset
for next_dataset in datasets_list:
    print(next_dataset)

    # Get subjects details
    subjects_file = os.path.join(eeg_datasets_path, next_dataset, 'participants.tsv')
    if not os.path.isfile(subjects_file):
        raise Exception(f'File not found: {subjects_file}')
    subjects_df = pd.read_csv(subjects_file, sep='\t')

    # Iterate through each subject in the dataset
    for _, subject_row in subjects_df.iterrows():
        # Get the subject features
        subject_features = {'subject_id': subject_row['participant_id'], 
                            'age': subject_row['AGE'], 'gender': subject_row['GENDER'], 'pd': subject_row['TYPE']}

        # Get the raw EEG data
        temp_EEG_raw = get_EEG_raw(next_dataset, subject_row['participant_id'])

        # Preprocess
        # TODO: Add filtering
        # TODO: Add ICA
        temp_EEG_clean = temp_EEG_raw

        # Power spectrum
        temp_spectra = get_spectra(temp_EEG_clean)
        
        # Spectral Param
        specparam_features = get_specparams(temp_spectra)

        # Add subject and component features to the features dataframe
        subject_features.update(specparam_features)
        study_features_df = pd.concat([study_features_df, pd.DataFrame([subject_features])], ignore_index=True)

del next_dataset, subjects_file, subjects_df, subject_row, subject_features, specparam_features, temp_EEG_raw, temp_EEG_clean, temp_spectra


ds004584-1.0.0
Reading /Users/stuartgow/GitHub/AI_Dissertation/EEG_Datasets_Source_x/ds004584-1.0.0/sub-001/eeg/sub-001_task-Rest_eeg.fdt
Reading 0 ... 140829  =      0.000 ...   281.658 secs...
Computing power spectrum using welch method...
Effective window size : 4.096 (s)
For Spectra Group:
- 63 channels & 1020 frequencies
- Frequency Range: 1 to 250 Hz
- Frequency resolution 0.24 Hz
Extracting Periodic & Aperiodic Components with Parameters:
- Peak width limits: [1, 12]
- Min peak height: 0.05
- Max peaks: 25
- Range: [1, 75]
- Aperiodic Mode: fixed


/var/folders/wd/rhzj_w8570g_y66t6j6mh4zw0000gn/T/ipykernel_89708/1201790303.py:26: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_dataset_raw = mne.io.read_raw_eeglab(eeg_lab_file_path, preload=True)


Running FOOOFGroup:   0%|          | 0/63 [00:00<?, ?it/s]

Reading /Users/stuartgow/GitHub/AI_Dissertation/EEG_Datasets_Source_x/ds004584-1.0.0/sub-002/eeg/sub-002_task-Rest_eeg.fdt
Reading 0 ... 163019  =      0.000 ...   326.038 secs...
Computing power spectrum using welch method...
Effective window size : 4.096 (s)
For Spectra Group:
- 63 channels & 1020 frequencies
- Frequency Range: 1 to 250 Hz
- Frequency resolution 0.24 Hz
Extracting Periodic & Aperiodic Components with Parameters:
- Peak width limits: [1, 12]
- Min peak height: 0.05
- Max peaks: 25
- Range: [1, 75]
- Aperiodic Mode: fixed


/var/folders/wd/rhzj_w8570g_y66t6j6mh4zw0000gn/T/ipykernel_89708/1201790303.py:26: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_dataset_raw = mne.io.read_raw_eeglab(eeg_lab_file_path, preload=True)


...

/var/folders/wd/rhzj_w8570g_y66t6j6mh4zw0000gn/T/ipykernel_89708/1201790303.py:26: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_dataset_raw = mne.io.read_raw_eeglab(eeg_lab_file_path, preload=True)


Running FOOOFGroup:   0%|          | 0/64 [00:00<?, ?it/s]

Reading /Users/stuartgow/GitHub/AI_Dissertation/EEG_Datasets_Source_x/ds004584-1.0.0/sub-149/eeg/sub-149_task-Rest_eeg.fdt
Reading 0 ... 64209  =      0.000 ...   128.418 secs...
Computing power spectrum using welch method...
Effective window size : 4.096 (s)
For Spectra Group:
- 64 channels & 1020 frequencies
- Frequency Range: 1 to 250 Hz
- Frequency resolution 0.24 Hz
Extracting Periodic & Aperiodic Components with Parameters:
- Peak width limits: [1, 12]
- Min peak height: 0.05
- Max peaks: 25
- Range: [1, 75]
- Aperiodic Mode: fixed


/var/folders/wd/rhzj_w8570g_y66t6j6mh4zw0000gn/T/ipykernel_89708/1201790303.py:26: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_dataset_raw = mne.io.read_raw_eeglab(eeg_lab_file_path, preload=True)


Running FOOOFGroup:   0%|          | 0/64 [00:00<?, ?it/s]